## Problem Statement
Find the minimum of $f(x) =\left(\frac{\sqrt{3}}{2} (x_1−3)+\frac{1}{2}(x_2−5)\right)^2+ 5 \cdot \left((\frac{\sqrt{3}}{2} (x_2−5)−\frac{1}{2} (x_1−3)\right)^2$ using GA.


In [1]:
import numpy as np
import math
def generate_population(param_size, pop_size, low_bound, high_bound, discrete = False):
    """
    Generates a random matrix of size (pop_size, param_size) with values in the given bounds. 
    Integer values are used if discrete = True
    """
    population = np.random.uniform(low= low_bound, high = high_bound, size=(pop_size, param_size))
    if discrete:
        population = np.round(population)
    return population

In [3]:
PARAM_SIZE = 2
POP_SIZE = 10
LOW_BOUND = -10
HIGH_BOUND = 10
population = generate_population(PARAM_SIZE, POP_SIZE, LOW_BOUND, HIGH_BOUND)
print(population.shape)
print(np.mean(population))

(10, 2)
-0.8407271833626163


In [8]:
def calculate_fitness(total_pop):
    """ Calculates fitness using the given function f(x) = 10 *x^2
    Returns a (POP_SIZE, ) numpy array
    """       
    fitness = np.zeros(total_pop.shape[0])
    for i in range(total_pop.shape[0]):
        x_1, x_2 = total_pop[i]
        factor1 = (math.sqrt(3)/2 *(x_1-3) + 0.5*(x_2-5))**2
        factor2 = 5*(math.sqrt(3)/2 *(x_2-5) - 0.5*(x_1-3))**2
        fitness[i] = factor1+ factor2
    return fitness

In [10]:
fitness = calculate_fitness(population)
print(fitness.shape)
print(fitness)

(10,)
[ 64.66146793  50.56221661 719.86676736 177.243344   485.23658621
  54.4358628  549.97426881 384.7292004   50.27422279  82.17014928]


In [12]:
def select_determinstic(t_pop, t_fitness, n_mating, n_params):
    """ Given current population t_pop, select n_mating number of parents using determinstic selection scheme
    based on t_fitness (the fitness of the population) 

    Returns parents and their fitness as a tuple (parent, fitness_of_parents)
    """    
    parents = np.empty((n_mating, n_params))
    
    best_fitness = np.argsort(t_fitness)[0:n_mating]
    parents = t_pop[best_fitness]
    return (parents, calculate_fitness(parents))

In [13]:
N_MATING = 4
parents, par_fitness = select_determinstic(population, fitness, N_MATING, PARAM_SIZE)
print(parents.shape)
print(parents)
print(par_fitness)

(4, 2)
[[-1.70560155  5.33917018]
 [ 7.12233394  4.07761783]
 [ 0.16034696  7.10042142]
 [-4.15504447  1.49307533]]
[50.27422279 50.56221661 54.4358628  64.66146793]


In [14]:
def crossover(t_parents, n_offspring, idx_crossover):
    """ Given a set of parents, combine them and return offspring vectors
    
    Returns the offsprings and their fitness
    """
    
    # Create an emppty vector
    offspring = np.empty((n_offspring, t_parents.shape[1]))

    # Fill in crossover details
    for i in range(n_offspring):
        parent1 = t_parents[i]
        parent2 = t_parents[i+1]
        offspring[i] = np.copy(parent2)
        offspring[i,idx_crossover:] = parent1[idx_crossover:]
        
    return (offspring, calculate_fitness(offspring))
def mutation(t_offspring, mutation_rate):
    """ Given a set of offsprings, introduce mutation in them
   
    Returns the mutated offsprings and their fitness
    """
    # Fill in details
    mutated_offspring = np.copy(t_offspring)
    
    random_values = np.random.uniform(low=-0.5, high=0.5, size=t_offspring.shape)
    coin_toss = np.random.uniform(low=0, high=1, size=t_offspring.shape)
    ind = np.nonzero(coin_toss > mutation_rate)
    print(ind)
    print(random_values.shape)
    mutated_offspring[ind] += random_values[ind]
        
    return (mutated_offspring, calculate_fitness(mutated_offspring))

In [15]:
N_OFFSPRING =  N_MATING-1 # Fill in how many offspring you want
IDX_CROSSOVER = 0 # Fill in at which index you want crossover
PM = 0.5 # Mutation rate parameter

offspring,  offspring_fitness = crossover(parents, N_OFFSPRING, IDX_CROSSOVER)
offspring, offspring_fitness = mutation(offspring, 0.5)
print(offspring)
print(offspring_fitness)

(array([2]), array([0]))
(3, 2)
[[-1.70560155  5.33917018]
 [ 7.12233394  4.07761783]
 [-0.12485457  7.10042142]]
[50.27422279 50.56221661 59.91318333]


In [16]:
def environmental_selection(curr_population, offspring):
    """ Calculate total population (after constraint checking) fitness,
    rank accoridingly and select only the top POP_SIZE individuals to
    pass on to the next generation
    """ 

    # Fill in details
    t_total_pop = np.vstack((curr_population, offspring))
    new_pop, new_pop_fitness = select_determinstic(t_total_pop, calculate_fitness(t_total_pop), curr_population.shape[0], curr_population.shape[1])
    return new_pop

In [17]:
new_pop = environmental_selection(population, offspring)
print(new_pop)

[[-1.70560155  5.33917018]
 [-1.70560155  5.33917018]
 [ 7.12233394  4.07761783]
 [ 7.12233394  4.07761783]
 [ 0.16034696  7.10042142]
 [-0.12485457  7.10042142]
 [-4.15504447  1.49307533]
 [-1.13022364 -0.68865452]
 [-7.48262253 -2.69690165]
 [ 1.77798018 -5.31258681]]


In [18]:
best_outputs = []
num_generations = 1000
curr_population = generate_population(PARAM_SIZE, POP_SIZE, LOW_BOUND, HIGH_BOUND)
overall_max_fitness = 99999

# Run many iterations
# You should also have another convergence check
for generation in range(num_generations):
    print("Generation : ", generation)

    # Measuring the fitness of each chromosome in the population.
    fitness = calculate_fitness(curr_population)
    print('Fitness')
    print(fitness.shape)
    print(fitness.T)

    min_fitness = np.min(fitness)

    # The best result in the current iteration.
    print("Best result in current iteration {0} compared to overall {1}".format(min_fitness, min(min_fitness, overall_max_fitness)))
    best_outputs.append(min_fitness)
    
    # Selecting the best parents in the population for mating.
    N_MATING = 6
    parents, par_fitness = select_determinstic(curr_population, fitness, N_MATING, PARAM_SIZE)
    print("Parents")
    print(parents.shape)
    print(parents)
    print(par_fitness)
#     parents, _ = select_stochastic(curr_population, fitness)
    
    # print("Parents")
    # print(parents)
    N_OFFSPRING =  N_MATING-1 # Fill in how many offspring you want
    IDX_CROSSOVER = 0 # Fill in at which index you want crossover
    PM = 0.5 # Mutation rate parameter

    offspring,  offspring_fitness = crossover(parents, N_OFFSPRING, IDX_CROSSOVER)
    offspring, offspring_fitness = mutation(offspring, PM)
    print("Mutation")
    print(offspring)
    print(offspring_fitness)

    # print("Mutation")
    # print(offspring_mutation)

    # Environmental selection
    curr_population = environmental_selection(curr_population, offspring)
              
# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = calculate_fitness(curr_population)
print(fitness)
# Then return the index of that solution corresponding to the best fitness.
max_idx = np.argmin(fitness)

print("Best solution : ", curr_population[max_idx, :])
print("Best solution fitness : ", fitness[max_idx])

Generation :  0
Fitness
(10,)
[546.55878762 167.09775447 500.86943188 199.69101488 359.44580947
 331.31880888 304.42469035 168.16337084 828.80163748  83.5546783 ]
Best result in current iteration 83.55467830401945 compared to overall 83.55467830401945
Parents
(6, 2)
[[ 3.33711432  9.71250029]
 [-2.8369892   8.05170528]
 [-1.66739915 -2.95679501]
 [-5.9585707  -3.86342582]
 [-5.44888617  8.67628968]
 [-1.06882616 -5.57412647]]
[ 83.5546783  167.09775447 168.16337084 199.69101488 304.42469035
 331.31880888]
(array([0, 0, 1, 1, 2, 2, 3, 3]), array([0, 1, 0, 1, 0, 1, 0, 1]))
(5, 2)
Mutation
[[ 3.68542149  9.90435787]
 [-2.54830715  8.22181296]
 [-2.03395163 -3.16338123]
 [-5.64848224 -3.6275292 ]
 [-5.44888617  8.67628968]]
[ 85.50575118 165.01066124 174.89048939 188.85547377 304.42469035]
Generation :  1
Fitness
(10,)
[ 83.5546783   85.50575118 165.01066124 167.09775447 168.16337084
 174.89048939 188.85547377 199.69101488 304.42469035 304.42469035]
Best result in current iteration 83.5546

# Results
After running the above code, we get:
```
Best solution :  [2.99936671 4.99958592]
Best solution fitness :  5.795615705510446e-07
```
Which is quite close to the correct solution of $x=\begin{pmatrix} 3 \\ 5\end{pmatrix} $ to minimize the function $f(x) =\left(\frac{\sqrt{3}}{2} (x_1−3)+\frac{1}{2}(x_2−5)\right)^2+ 5 \cdot \left((\frac{\sqrt{3}}{2} (x_2−5)−\frac{1}{2} (x_1−3)\right)^2$